# Testing around with the Filter Bank and SVM Approach

In [2]:
import numpy as np
import mne
import pickle
from mne.preprocessing import read_ica
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.svm import SVC, SVR
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from mne.decoding import CSP, SPoC
from continuous_control_bci.util import emg_classes_to_eeg_classes, SUBJECT_IDS
from tqdm import tqdm
import matplotlib as plt

from continuous_control_bci.data.load_data import load_calibration, load_driving
from continuous_control_bci.data.preprocessing import make_epochs, epochs_to_train_test
from continuous_control_bci.modelling.csp_classifier import create_csp_classifier, get_driving_epochs_for_csp
from sklearn.metrics import f1_score
from filterbank_csp import filter_bank, filter_bank_csp, predict_against_threshold, predict_against_threshold_indiv, create_csp_svm_classifier, majority_pred
from scipy.signal import cheby2, sosfiltfilt

In [2]:
# Subject 61 should have fairly good EEG data and performance
subject_id = "061"
include_rest = True
raw = load_calibration(subject_id)
raw.set_eeg_reference()
raw.filter(l_freq=5, h_freq=35)
eeg_channel_indices = mne.pick_types(raw.info, eeg=True, emg=False)
ica = read_ica(f'D:/RUG/Term 2/FYRP/data/ica/P{subject_id}-calibration-ica.fif')
ica.apply(raw)

epochs = make_epochs(raw, include_rest=include_rest)

X_train, _, y_train, _ = epochs_to_train_test(epochs)
print("Training classifier. This may take a while..")
rank = {
    'eeg': X_train.shape[1] - len(ica.exclude),
    'mag': 32,
}


Extracting EDF parameters from D:\RUG\Term 2\FYRP\data\ivo_data\sub-P061\motor-imagery-csp-061-acquisition-[2024.02.07-14.35.15].gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
Channel 1, Channel 2, Channel 3, Channel 4, Channel 5, Channel 6, Channel 7, Channel 8, Channel 9, Channel 10, Channel 11, Channel 12, Channel 13, Channel 14, Channel 15, Channel 16, Channel 17, Channel 18, Channel 19, Channel 20, Channel 21, Channel 22, Channel 23, Channel 24, Channel 25, Channel 26, Channel 27, Channel 28, Channel 29, Channel 30, Channel 31, Channel 32, EX 1, EX 2, EX 3, EX 4, EX 5, EX 6, EX 7, EX 8
Creating raw.info structure...
Reading 0 ... 882783  =      0.000 ...   431.046 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 5 - 35 Hz

FIR filter paramete

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Reading D:/RUG/Term 2/FYRP/data/ica/P061-calibration-ica.fif ...
Now restoring ICA solution ...
Ready.
Applying ICA to Raw instance


[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    0.3s finished


    Transforming to ICA space (32 components)
    Zeroing out 2 ICA components
    Projecting back using 32 PCA components
Used Annotations descriptions: ['769', '770']
Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 40 events and 7681 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['800']
Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 40 events and 7681 original time points ...
0 bad epochs dropped
Not setting metadata
80 matching events found
No baseline correction applied
Training classifier. This may take a while..


D:\RUG\Term 2\FYRP\continuous_control_bci\continuous_control_bci\data\preprocessing.py:83: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs([epochs, rest_epochs], add_offset=False)
D:\RUG\Term 2\FYRP\continuous_control_bci\continuous_control_bci\data\preprocessing.py:83: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epochs = mne.concatenate_epochs([epochs, rest_epochs], add_offset=False)


## Out of interest I started by just using the individual frequency bands and trained a classifier on each of them to see training accuracies

In [5]:
# Function for filtering the signal, freq bands picked are
freq_bands = [[5,8],[5,12],[8,12],[10,14],[12,16],[13,30]]
filterd_calib = filter_bank(epochs.get_data(picks=eeg_channel_indices),freq_bands=freq_bands, order=4)
# simply using the individual freq-ranges to make predictions
y_preds = []
clfs = []
for data in filterd_calib:
    clf, y_pred = create_csp_classifier(data, y_train, rank)
    clfs.append(clf)
    y_preds.append(y_pred)
    print(classification_report(y_train, y_pred, target_names=["Left", "Right", "Rest"]))


IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 5.00, 8.00 Hz: -20.00, -20.00 dB


IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 5.00, 12.00 Hz: -20.00, -20.00 dB


IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 8.00, 12.00 Hz: -20.00, -20.00 dB


IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 10.00, 14.00 Hz: -20.00, -20.00 dB


IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (t

# Now trying to combine the CSP featrues and then select the most informative ones

In [6]:
# using the average_power transform into we get one value for the whole epoch feature channel, we chose 6 frequency bands and 6 components = 36
csp_channels, csp_info = filter_bank_csp(X_train, y_train, freq_bands, 4, rank)



IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 5.00, 8.00 Hz: -20.00, -20.00 dB

Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.

IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 5.0

array([[-0.82050273, -0.32822082, -1.02903227, ..., -0.43396845,
        -0.61082532, -0.20972365],
       [-0.19879088, -1.01559918,  0.21325659, ..., -0.46880308,
        -0.63132953, -0.443927  ],
       [ 0.52025979, -0.45348079,  0.17834498, ..., -0.54662501,
        -0.86423456,  0.02998544],
       ...,
       [-0.6986843 , -0.6840568 , -0.01848055, ...,  0.08764144,
         1.78820574, -0.03605475],
       [-0.00641142, -0.75818478, -0.79975566, ..., -0.49518002,
         0.30145204,  0.24704696],
       [-0.45452203,  0.06215081,  0.04604964, ...,  0.20406627,
         1.57326341,  0.10517648]])

In [8]:
# Training and applying the Logistic Regression with all 36 features (too many)
clf = LogisticRegression()
y_pred_fbcsp = cross_val_predict(clf, csp_channels, y_train, cv=5)
print(classification_report(y_train, y_pred_fbcsp, target_names=["Left", "Right", "Rest"]))

              precision    recall  f1-score   support

        Left       1.00      0.85      0.92        20
       Right       0.90      0.95      0.93        20
        Rest       0.93      0.97      0.95        40

    accuracy                           0.94        80
   macro avg       0.94      0.92      0.93        80
weighted avg       0.94      0.94      0.94        80



In [10]:
# Using mutual information to find the most informative 10 channels
sel_K = SelectKBest(mutual_info_classif, k=10).fit(csp_channels,y_train)
new_train = sel_K.transform(csp_channels)
# Training a LogisticRegression algo again
clf = LogisticRegression()
y_pred_fbcsp = cross_val_predict(clf, new_train, y_train, cv=5)
clf.fit(new_train, y_train)
print(classification_report(y_train, y_pred_fbcsp, target_names=["Left", "Right", "Rest"]))

              precision    recall  f1-score   support

        Left       1.00      0.95      0.97        20
       Right       0.95      1.00      0.98        20
        Rest       1.00      1.00      1.00        40

    accuracy                           0.99        80
   macro avg       0.98      0.98      0.98        80
weighted avg       0.99      0.99      0.99        80



In [11]:
# doing the same with a SVC
clf = SVC()
y_pred_fbcsp = cross_val_predict(clf, csp_channels, y_train, cv=5)
print(classification_report(y_train, y_pred_fbcsp, target_names=["Left", "Right", "Rest"]))

              precision    recall  f1-score   support

        Left       1.00      0.90      0.95        20
       Right       0.95      1.00      0.98        20
        Rest       0.98      1.00      0.99        40

    accuracy                           0.97        80
   macro avg       0.98      0.97      0.97        80
weighted avg       0.98      0.97      0.97        80



In [17]:
svm_clas = SVC()
y_pred_fbcsp = cross_val_predict(svm_clas, new_train, y_train, cv=5)
svm_clas.fit(new_train, y_train) #for later use
print(classification_report(y_train, y_pred_fbcsp, target_names=["Left", "Right", "Rest"]))

              precision    recall  f1-score   support

        Left       1.00      0.95      0.97        20
       Right       0.95      1.00      0.98        20
        Rest       1.00      1.00      1.00        40

    accuracy                           0.99        80
   macro avg       0.98      0.98      0.98        80
weighted avg       0.99      0.99      0.99        80



# Now apply the filters and predictor to the driving Data
 - 200ms time windows, no overlap
 - First Filtering, then CSP then predicting

In [14]:
driving = load_driving(subject_id)
driving.raw.set_eeg_reference()
driving.raw.filter(l_freq=5, h_freq=35)
ica.apply(driving.raw)

# EEG Data Stream
eeg_channel_indices = mne.pick_types(raw.info, eeg=True, emg=False)
eeg_data, eeg_time = driving.raw[eeg_channel_indices]
# Those windows are basically my epochs on which I try to predict something, if a certain amount of windows have the same prediction (3.5 sec or smth) this can then be used to controll the car
eeg_windows = mne.make_fixed_length_epochs(driving.raw, duration=0.2, preload=True, reject_by_annotation=False)

X_driving = eeg_windows.get_data(copy=False, picks=eeg_channel_indices)

# Making predictions based on EEG windows
# -1 left, 0 rest, 1 right
y_driving = driving.emg_prediction_stream['time_series']
y_driving = emg_classes_to_eeg_classes(y_driving)
y_driving = np.squeeze(y_driving)
print(X_driving.shape)
print(y_driving[:-1].shape)

Creating RawArray with float64 data, n_channels=40, n_times=5494682
    Range : 0 ... 5494681 =      0.000 ...  2682.950 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


D:\RUG\Term 2\FYRP\continuous_control_bci\continuous_control_bci\data\load_data.py:28: RuntimeWarning: The unit for channel(s) AF3, AF4, C3, C4, CP1, CP2, CP5, CP6, Cz, F3, F4, F7, F8, FC1, FC2, FC5, FC6, Fp1, Fp2, Fz, LHEOG, LVEOG, Left-extension, Left-flexion, O1, O2, Oz, P3, P4, P7, P8, PO3, PO4, Pz, RHEOG, Right-extension, Right-flexion, T7, T8, UVEOG has changed from NA to V.
  streams.raw.set_channel_types(CHANNEL_TYPE_MAPPING)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 5 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 3381 samples (1.651 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    4.1s finished


Applying ICA to Raw instance
    Transforming to ICA space (32 components)
    Zeroing out 2 ICA components
    Projecting back using 32 PCA components
Not setting metadata
13414 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 13414 events and 410 original time points ...
0 bad epochs dropped
(13414, 32, 410)
(13414,)


In [ ]:
driving_filtered = filter_bank(X_driving, freq_bands, 4)
driving_csp = []
for i in range(driving_filtered.shape[0]):
    driving_ = csp_info[i].transform(driving_filtered[i])
    if i == 0:
        driving_csp = driving_
    else:
        driving_csp = np.concatenate((driving_csp, driving_), axis=1)
driving_csp = sel_K.transform(driving_csp)

In [19]:
driving_y_pred = svm_clas.predict(driving_csp)
print(classification_report(y_driving[:-1], driving_y_pred, target_names=["Left", "Right", "Rest"]))

              precision    recall  f1-score   support

        Left       0.27      0.04      0.07      4338
       Right       0.00      0.00      0.00      2923
        Rest       0.45      0.93      0.61      6153

    accuracy                           0.44     13414
   macro avg       0.24      0.32      0.23     13414
weighted avg       0.29      0.44      0.30     13414



# Doing it for all Participants

In [3]:
freq_bands = [[4,8],[4,12],[8,12],[10,14],[12,16],[13,30]]
for subject_id in SUBJECT_IDS:
    # Subject 61 should have fairly good EEG data and performance
    include_rest = True
    raw = load_calibration(subject_id)
    raw.set_eeg_reference()
    raw.filter(l_freq=5, h_freq=35)
    eeg_channel_indices = mne.pick_types(raw.info, eeg=True, emg=False)
    ica = read_ica(f'D:/RUG/Term 2/FYRP/data/ica/P{subject_id}-calibration-ica.fif')
    ica.apply(raw)

    epochs = make_epochs(raw, include_rest=include_rest)

    X_train, _, y_train, _ = epochs_to_train_test(epochs)
    print("Training classifier. This may take a while..")
    rank = {
        'eeg': X_train.shape[1] - len(ica.exclude),
        'mag': 32,
    }
    csp_channels, csp_info = filter_bank_csp(X_train, y_train, freq_bands, 4, rank)
    # Using mutual information to find the most informative 10 channels
    sel_K = SelectKBest(mutual_info_classif, k=10).fit(csp_channels,y_train)
    new_train = sel_K.transform(csp_channels)

    svm_clas = SVC()
    y_pred_fbcsp = cross_val_predict(svm_clas, new_train, y_train, cv=5)
    svm_clas.fit(new_train, y_train) #for later use
    report = classification_report(y_train, y_pred_fbcsp, target_names=["Left", "Right", "Rest"], output_dict=True)
    print(classification_report(y_train, y_pred_fbcsp, target_names=["Left", "Right", "Rest"]))

    with open(f"data/calib_report_fb_{subject_id}.pkl", "wb") as fp:
        pickle.dump(report, fp)

    driving = load_driving(subject_id)
    driving.raw.set_eeg_reference()
    driving.raw.filter(l_freq=5, h_freq=35)
    ica.apply(driving.raw)

    # EEG Data Stream
    eeg_data, eeg_time = driving.raw[eeg_channel_indices]
    # Those windows are basically my epochs on which I try to predict something, if a certain amount of windows have the same prediction (3.5 sec or smth) this can then be used to controll the car
    eeg_windows = mne.make_fixed_length_epochs(driving.raw, duration=0.2, preload=True, reject_by_annotation=False)

    X_driving = eeg_windows.get_data(copy=False, picks=eeg_channel_indices)

    # Making predictions based on EEG windows
    # -1 left, 0 rest, 1 right
    y_driving = driving.emg_prediction_stream['time_series']
    y_driving = emg_classes_to_eeg_classes(y_driving)
    y_driving = np.squeeze(y_driving)
    # TODO: Check if the timestamps indicate whether to remove beginning or end of the array
    if X_driving.shape[0] == y_driving.shape[0]:
        pass
    elif X_driving.shape[0] > y_driving.shape[0]:
        diff = X_driving.shape[0]-y_driving.shape[0]
        X_driving = X_driving[:-diff,:,:]
    else:
        diff = y_driving.shape[0]-X_driving.shape[0]
        y_driving = y_driving[:-diff]

    driving_filtered = filter_bank(X_driving, freq_bands, 4)

    driving_csp = []
    for i in range(driving_filtered.shape[0]):
        driving_ = csp_info[i].transform(driving_filtered[i])
        if i == 0:
            driving_csp = driving_
        else:
            driving_csp = np.concatenate((driving_csp, driving_), axis=1)
    driving_csp = sel_K.transform(driving_csp)

    driving_y_pred = svm_clas.predict(driving_csp)
    report = classification_report(y_driving, driving_y_pred, target_names=["Left", "Right", "Rest"], output_dict=True)
    print(classification_report(y_driving, driving_y_pred, target_names=["Left", "Right", "Rest"]))

    with open(f"data/driving_report_fb_{subject_id}.pkl", "wb") as fp:
        pickle.dump(report, fp)


Extracting EDF parameters from D:\RUG\Term 2\FYRP\data\ivo_data\sub-P061\motor-imagery-csp-061-acquisition-[2024.02.07-14.35.15].gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
Channel 1, Channel 2, Channel 3, Channel 4, Channel 5, Channel 6, Channel 7, Channel 8, Channel 9, Channel 10, Channel 11, Channel 12, Channel 13, Channel 14, Channel 15, Channel 16, Channel 17, Channel 18, Channel 19, Channel 20, Channel 21, Channel 22, Channel 23, Channel 24, Channel 25, Channel 26, Channel 27, Channel 28, Channel 29, Channel 30, Channel 31, Channel 32, EX 1, EX 2, EX 3, EX 4, EX 5, EX 6, EX 7, EX 8
Creating raw.info structure...
Reading 0 ... 882783  =      0.000 ...   431.046 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 5 - 35 Hz

FIR filter paramete

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Reading D:/RUG/Term 2/FYRP/data/ica/P061-calibration-ica.fif ...
Now restoring ICA solution ...
Ready.
Applying ICA to Raw instance


[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    0.4s finished


    Transforming to ICA space (32 components)
    Zeroing out 2 ICA components
    Projecting back using 32 PCA components
Used Annotations descriptions: ['769', '770']
Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 40 events and 7681 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['800']
Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 40 events and 7681 original time points ...
0 bad epochs dropped
Not setting metadata
80 matching events found
No baseline correction applied
Training classifier. This may take a while..

IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 8.00 Hz: -20.00, -20.00 dB


...

D:\RUG\Term 2\FYRP\continuous_control_bci\continuous_control_bci\data\load_data.py:28: RuntimeWarning: The unit for channel(s) AF3, AF4, C3, C4, CP1, CP2, CP5, CP6, Cz, F3, F4, F7, F8, FC1, FC2, FC5, FC6, Fp1, Fp2, Fz, LHEOG, LVEOG, Left-extension, Left-flexion, O1, O2, Oz, P3, P4, P7, P8, PO3, PO4, Pz, RHEOG, Right-extension, Right-flexion, T7, T8, UVEOG has changed from NA to V.
  streams.raw.set_channel_types(CHANNEL_TYPE_MAPPING)


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 5 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 3381 samples (1.651 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    3.9s finished


Applying ICA to Raw instance
    Transforming to ICA space (32 components)
    Zeroing out 4 ICA components
    Projecting back using 32 PCA components
Not setting metadata
12967 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 12967 events and 410 original time points ...
0 bad epochs dropped

IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 8.00 Hz: -20.00, -20.00 dB


IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 12.00 Hz: -20.00, -20.00 dB


IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backwar

D:\Artificial Intelligence\Bac Thesis\Thesis_code\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Artificial Intelligence\Bac Thesis\Thesis_code\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Artificial Intelligence\Bac Thesis\Thesis_code\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Either im doing smth wrong or we overfit heavily here. Seeing that it works fine with the training data i assume second. How to fix this?

# Thoughts on Interleaving EMG and EEG to learn to use EEG better on the fly:
    - Make both EMG and EEG predictions and compare them at every timestep
    - if they are different from each other for multiple steps (finding a threshold that is good) we use EMG if they are the same use EEG




In [6]:
for subject_id in SUBJECT_IDS:
    with open(f"data/driving_report_fb_{subject_id}.pkl", "rb") as fp:
        report = pickle.load(fp)
        print(f"Subject {subject_id}")
        print("Left f1-score", report['Left']["f1-score"])
        print("Right f1-score", report['Right']["f1-score"])
        print("Rest f1-score", report['Rest']["f1-score"])
        print("Accuracy f1-score", report['accuracy'])
        print("\n")

Subject 061
Left f1-score 0.05835866261398176
Right f1-score 0.0
Rest f1-score 0.6075735138565687
Accuracy f1-score 0.4389443864619055


Subject 066
Left f1-score 0.0
Right f1-score 0.0
Rest f1-score 0.6471068976789801
Accuracy f1-score 0.47831339857436267


Subject 126
Left f1-score 0.0
Right f1-score 0.39390073507077333
Rest f1-score 0.0
Accuracy f1-score 0.2452530448596717


Subject 280
Left f1-score 0.031061259706643658
Right f1-score 0.0
Rest f1-score 0.6292525707269658
Accuracy f1-score 0.4611279922922997


Subject 302
Left f1-score 0.0
Right f1-score 0.0
Rest f1-score 0.5945799163634389
Accuracy f1-score 0.4230620604374372


Subject 336
Left f1-score 0.0
Right f1-score 0.009430784776018861
Rest f1-score 0.4887011928576721
Accuracy f1-score 0.3240070436313833


Subject 381
Left f1-score 0.26903283052351373
Right f1-score 0.0007339449541284401
Rest f1-score 0.6145359417652411
Accuracy f1-score 0.4750231267345051


Subject 406
Left f1-score 0.00154320987654321
Right f1-score 0.0099